In [1]:
pip install tensorflow keras emnist numpy matplotlib


In [2]:
import numpy as np
from emnist import extract_training_samples, extract_test_samples
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle

In [4]:
import shutil
shutil.rmtree('/root/.emnist', ignore_errors=True)


In [5]:
!pip install --upgrade emnist


In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

ds_train = tfds.load('emnist/balanced', split='train', as_supervised=True)
ds_test = tfds.load('emnist/balanced', split='test', as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.0LD9PV_3.1.0/emnist-train.tfrecord*...:   0%|  …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.0LD9PV_3.1.0/emnist-test.tfrecord*...:   0%|   …

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/balanced/3.1.0. Subsequent calls will reuse this data.


In [11]:
pip install tensorflow-datasets


In [12]:
import tensorflow_datasets as tfds
import numpy as np

# load dataset
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/balanced',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

num_classes = ds_info.features['label'].num_classes
print("Classes:", num_classes)


Classes: 47


In [13]:
import tensorflow as tf

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, -1)   # (28,28,1)
    return image, tf.one_hot(label, num_classes)

train_ds = ds_train.map(preprocess).batch(256).prefetch(1)
test_ds = ds_test.map(preprocess).batch(256).prefetch(1)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    MaxPooling2D(2,2),

    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.3),

    Dense(num_classes,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


Epoch 1/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 97s 213ms/step - accuracy: 0.4857 - loss: 1.8600 - val_accuracy: 0.8174 - val_loss: 0.5505
Epoch 2/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 136s 200ms/step - accuracy: 0.7992 - loss: 0.6228 - val_accuracy: 0.8413 - val_loss: 0.4603
Epoch 3/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 138s 193ms/step - accuracy: 0.8268 - loss: 0.5195 - val_accuracy: 0.8546 - val_loss: 0.4209
Epoch 4/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 138s 184ms/step - accuracy: 0.8416 - loss: 0.4675 - val_accuracy: 0.8614 - val_loss: 0.3985
Epoch 5/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 81s 185ms/step - accuracy: 0.8500 - loss: 0.4342 - val_accuracy: 0.8661 - val_loss: 0.3885
Epoch 6/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 82s 185ms/step - accuracy: 0.8582 - loss: 0.4079 - val_accuracy: 0.8677 - val_loss: 0.3790
Epoch 7/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 81s 184ms/step - accuracy: 0.8632 - loss: 0.3873 - val_accuracy: 0.8709 - val_loss: 0.3715
Epoch 8/10
441/441 ━━━━━━━━━━━━━━━━━━━━ 84s 189ms/step - accuracy: 0.8700 - loss

In [16]:
model.save("emnist_cnn_model.h5")


In [17]:
import pickle

label_map = ds_info.features['label'].names

with open("label_map.pkl","wb") as f:
    pickle.dump(label_map,f)

print(label_map[:10])


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
